In [1]:
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
#path = f"x264/sampledConfigurations_{name_of_technique}_t3.csv"
#lst = ['distBased', 'divDistBased', 'henard', 'random', 'solverBased', 'twise']

path = r"sampledConfigurations_divDistBased_t3.csv"


df = pd.read_csv(path, sep=';')


In [3]:
df

,root,HAVE_CRYPTO,HAVE_HASH,HAVE_REPLICATION,HAVE_VERIFY,HAVE_SEQUENCE,HAVE_STATISTICS,DIAGNOSTIC,PAGESIZE,PS1K,PS4K,PS8K,PS16K,PS32K,CACHESIZE,CS32MB,CS16MB,CS64MB,CS512MB,Performance
0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0.502896
1,1,0,0,1,1,1,1,1,1,0,1,0,0,0,1,1,0,0,0,0.478704
2,1,1,1,1,1,1,1,1,1,0,1,0,0,0,1,1,0,0,0,5.845517
3,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0.478544
4,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,10.756226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,1,1,1,1,1,0,1,1,1,0,1,0,0,0,1,1,0,0,0,5.840267
359,1,1,1,1,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,5.835302
360,1,1,0,1,1,1,1,0,1,0,1,0,0,0,1,1,0,0,0,6.494474
361,1,1,1,1,1,1,1,0,1,0,1,0,0,0,1,1,0,0,0,5.832149


### Geração de feature importance sem a utilização de hiperparâmetros

In [4]:
def run_featureSelection(df, hyperparameters=None, path=None):
    X = df.drop('Performance', axis=1)
    y = df['Performance']

    if hyperparameters is None:
        rf = RandomForestRegressor()
    else:
        rf = RandomForestRegressor(**hyperparameters)
    
    rf.fit(X, y)

    importances = rf.feature_importances_

    feature_importances = pd.DataFrame(importances, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)

    print(feature_importances)
    
    if hyperparameters is None:
        if path is None:
            feature_importances.to_csv("feature_importance.csv", header=True)
        else:
            feature_importances.to_csv("feature_importance_"+path+".csv", header=True)
    else:
        if path is None:
            feature_importances.to_csv("feature_importanceHP.csv", header=True)
        else:
            feature_importances.to_csv("feature_importanceHP_"+path+".csv", header=True)


run_featureSelection(df)

                    importance
PS32K             4.614398e-01
HAVE_CRYPTO       3.958522e-01
PS16K             1.151308e-01
PS8K              2.198484e-02
PS1K              2.592798e-03
PS4K              1.715005e-03
HAVE_HASH         1.273604e-03
DIAGNOSTIC        4.921959e-06
HAVE_STATISTICS   2.036466e-06
HAVE_SEQUENCE     8.775342e-07
HAVE_VERIFY       8.031347e-07
HAVE_REPLICATION  5.888029e-07
CS32MB            5.782818e-07
CS64MB            4.317765e-07
CS512MB           4.247353e-07
CS16MB            3.033876e-07
CACHESIZE         0.000000e+00
PAGESIZE          0.000000e+00
root              0.000000e+00


### Geração de feature importance com a utilização de hiperparâmetros

In [5]:
hyperparams_RF = {
    "max_depth": 25,
    "n_estimators": 10,
    "n_jobs" : -1
}

run_featureSelection(df, hyperparams_RF)

                    importance
PS32K             4.565638e-01
HAVE_CRYPTO       3.995137e-01
PS16K             1.166752e-01
PS8K              2.198451e-02
PS4K              2.393372e-03
PS1K              1.592265e-03
HAVE_HASH         1.265923e-03
DIAGNOSTIC        5.410926e-06
HAVE_STATISTICS   2.209318e-06
HAVE_SEQUENCE     9.657532e-07
HAVE_VERIFY       6.745998e-07
CS64MB            6.152990e-07
HAVE_REPLICATION  3.860616e-07
CS512MB           3.725141e-07
CS16MB            3.185953e-07
CS32MB            2.626470e-07
CACHESIZE         0.000000e+00
PAGESIZE          0.000000e+00
root              0.000000e+00


In [6]:
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

for config_name in samples_config:
    for t in [1,2,3]:
        path = r"sampledConfigurations_"+config_name+"_t"+str(t)+".csv"
        path_name = config_name+"_t"+str(t)
        df = pd.read_csv(path, sep=';')
        
        run_featureSelection(df, None, path_name)
        run_featureSelection(df, hyperparams_RF, path_name)

                    importance
PS16K             7.097421e-01
PS8K              1.428961e-01
PS1K              6.708580e-02
PS4K              4.012767e-02
HAVE_REPLICATION  1.285228e-02
HAVE_HASH         1.162079e-02
HAVE_CRYPTO       9.639011e-03
CS64MB            2.037091e-03
HAVE_STATISTICS   1.450727e-03
HAVE_SEQUENCE     9.063271e-04
DIAGNOSTIC        8.973192e-04
CS16MB            4.040535e-04
HAVE_VERIFY       1.772469e-04
CS32MB            1.634717e-04
CS512MB           1.658440e-08
CACHESIZE         0.000000e+00
PS32K             0.000000e+00
PAGESIZE          0.000000e+00
root              0.000000e+00
                    importance
PS16K             7.748487e-01
PS1K              1.046553e-01
PS8K              6.158375e-02
HAVE_SEQUENCE     2.249880e-02
PS4K              2.052233e-02
HAVE_HASH         9.989326e-03
CS32MB            4.227485e-03
DIAGNOSTIC        5.034896e-04
HAVE_STATISTICS   4.743368e-04
CS64MB            3.397340e-04
CS16MB            1.785150e-04
HAVE_VER

                    importance
HAVE_CRYPTO       8.833575e-01
HAVE_HASH         1.148833e-01
PS32K             1.211309e-03
DIAGNOSTIC        2.540662e-04
PS4K              1.225978e-04
PS16K             5.830995e-05
PS1K              4.490188e-05
HAVE_REPLICATION  1.698220e-05
HAVE_VERIFY       1.668376e-05
HAVE_STATISTICS   1.376983e-05
HAVE_SEQUENCE     7.547290e-06
PS8K              7.454867e-06
CS32MB            5.052981e-06
CS512MB           3.163073e-07
CS16MB            2.101969e-07
CACHESIZE         0.000000e+00
PAGESIZE          0.000000e+00
CS64MB            0.000000e+00
root              0.000000e+00
                    importance
HAVE_CRYPTO       8.901563e-01
HAVE_HASH         1.082747e-01
PS32K             1.075962e-03
DIAGNOSTIC        2.375098e-04
PS4K              1.016656e-04
PS16K             5.962327e-05
PS8K              2.026177e-05
PS1K              1.847227e-05
HAVE_REPLICATION  1.515163e-05
HAVE_STATISTICS   1.428580e-05
HAVE_VERIFY       1.300615e-05
HAVE_SEQ

                    importance
HAVE_CRYPTO       8.731187e-01
HAVE_HASH         1.264484e-01
DIAGNOSTIC        3.647872e-04
HAVE_VERIFY       2.037812e-05
HAVE_STATISTICS   1.822637e-05
HAVE_REPLICATION  1.371775e-05
HAVE_SEQUENCE     9.808680e-06
CS32MB            2.505132e-06
CS512MB           1.705168e-06
CS64MB            1.582458e-06
CS16MB            2.356198e-07
PS32K             0.000000e+00
CACHESIZE         0.000000e+00
root              0.000000e+00
PS16K             0.000000e+00
PS8K              0.000000e+00
PS4K              0.000000e+00
PAGESIZE          0.000000e+00
PS1K              0.000000e+00
                    importance
HAVE_CRYPTO       8.686996e-01
HAVE_HASH         1.308687e-01
DIAGNOSTIC        3.617395e-04
HAVE_VERIFY       2.211201e-05
HAVE_STATISTICS   1.883989e-05
HAVE_REPLICATION  1.376222e-05
HAVE_SEQUENCE     9.886395e-06
CS32MB            2.237134e-06
CS64MB            1.528093e-06
CS512MB           1.286577e-06
CS16MB            2.563775e-07
PS32K   